<a href="https://colab.research.google.com/github/Nicocoding/Selfbanksheet/blob/main/Selbank_Statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!/usr/share/locales/install-language-pack es_ES
!dpkg-reconfigure locales
import os
os.kill(os.getpid(), 9)

In [2]:
from requests import check_compatibility
import pandas as pd 
import yfinance as yf 
import locale
from google.colab import auth
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

auth.authenticate_user()

import gspread 
from google.auth import default 
creds, _ = default()
gc = gspread.authorize(creds)

sh = gc.open('Selfbank')
wks = sh.worksheet('Selfbank')
rows = wks.get_all_values()
dfrev = pd.DataFrame(rows)
New_Header = dfrev.iloc[1:]
dfrev = dfrev[2:]
dfrev.columns = New_Header.values
print(dfrev)
dfrev['Fecha'] = pd.to_datetime(dfrev['Fecha'], dayfirst = True)
dfrev['Fecha'] = dfrev['Fecha'].dt.date
dfrev = dfrev.sort_values(by="Fecha")
dfrev.reset_index(inplace=True, drop=True)
print(dfrev)

sh=gc.open('Logfile')
wks = sh.worksheet('Logfile')
rows = wks.get_all_values()
dflog = pd.DataFrame(rows)
New_Header = dflog.iloc[0]
dflog = dflog[1:]
dflog.columns = New_Header.values
dflog['Date'] = pd.to_datetime(dflog['Date'], dayfirst = True)
dflog['Date'] = dflog['Date'].dt.date

chop = dflog.iloc[-1,0]
dfrev = dfrev[~(dfrev['FECHA'] < chop)]
dfrev = dfrev[dfrev['OPERACIÓN'] != 'BAJA DERECHOS']
dfrev = dfrev[dfrev['OPERACIÓN'] != 'GENERA DERECHOS']
dfrev.reset_index(inplace=True, drop=True)

wks = sh.worksheet('Sheet1')
wks2 = sh.worksheet('Portfolio')
rows2 =wks2.get_all_values()
portfolio = pd.DataFrame(rows2)
New_Header = portfolio.iloc[0]
portfolio = portfolio[1:]
portfolio.columns = New_Header.values

wks2 = sh.worksheet('Taxes')
rows2 =wks2.get_all_values()
taxes = pd.DataFrame(rows2)
New_Header = taxes.iloc[0]
taxes = taxes[1:]
taxes.columns = New_Header.values

for index, rows in dfrev.iterrows():

  #Date
  date = rows['FECHA']
  datemod = date.strftime("%d/%m/%Y")
  wks.update_cell(index + 2, 1, datemod)

  #Type of transaction
  typeTRX = rows['OPERACIÓN']
  if typeTRX == 'COMPRA': 
    typeTRX = 'BUY'
    wks.update_cell(index + 2, 5, typeTRX)   
  if typeTRX == 'DIVIDENDO': 
    typeTRX = 'DIVIDEND'
    wks.update_cell(index + 2, 5, typeTRX)
  if typeTRX == 'VENTA': 
    typeTRX = 'SELL'
    wks.update_cell(index + 2, 5, typeTRX)

  #Name
  name = rows['VALOR']
  if name[:5] == "TEF.D":
    name = "TELEFONICA"
  wks.update_cell(index + 2, 3, name)

  #Ticker
  wks2 = sh.worksheet('tickers')
  rows2 =wks2.get_all_values()
  tickers = pd.DataFrame(rows2)
  New_Header = tickers.iloc[0]
  tickers = tickers[1:]
  tickers.columns = New_Header.values
  tickersupd = tickers.loc[tickers['Name'] == name]
  row = len(tickers.index)
  if tickersupd.empty:
    ticker = input('What is the ticker of the company ' + name + '?\n')
    print(type(ticker))
    wks2.update_cell(2 + row, 1, ticker)
    wks2.update_cell(2 + row, 2, name)
  else:
    ticker = tickersupd.iat[0,0]
  wks.update_cell(index + 2, 2, ticker)

  #Quote type
  tick = yf.Ticker(ticker)
  quoteType = tick.info['quoteType']
  wks.update_cell(index + 2, 4, quoteType)

  #Currency
  ccy = tick.info['currency']
  wks.update_cell(index + 2, 6, ccy)

  #Exchange rate with respect to EUR
  if ccy == "EUR":
    hist = 1
  else:
    tickccy = "EUR" + ccy + "=X"
    tickccy = yf.Ticker(tickccy)
    hist = tickccy.history(start=date, end=date)
    hist = hist["Close"].iloc[0]
    hist = round(hist,4)
  wks.update_cell(index + 2, 7, hist)

  #Quantity
  quantity = locale.atof(rows['TÍTULOS'])
  wks.update_cell(index + 2, 8, quantity)

  #Country
  country = tick.info['country']
  countryupd = taxes.loc[taxes['Country'] == country]
  if countryupd.empty: 
    tax = "19%"
  else:
    tax = countryupd.iloc[0]["Final tax rate"]
  wks.update_cell(index + 2, 12, tax)
  wks.update_cell(index + 2, 13, country)

  #Total
  total = locale.atof(rows['IMPORTE TOTAL (€)'])
  wks.update_cell(index + 2, 11, total)

  #Fees & price
  pricedo = locale.atof(rows['PRECIO EN DIVISA ORIGEN'])
  if typeTRX == 'DIVIDEND':
    fees = 0
    tax = locale.atof(tax.strip('%'))/10000
    price = (total / quantity)/(1-tax)
    wks.update_cell(index + 2, 9, price)
    wks.update_cell(index + 2, 10, fees)
  else:
    price = pricedo/hist
    fees = abs(price*quantity - total)
    wks.update_cell(index + 2, 9, price)
    wks.update_cell(index + 2, 10, fees)

  #Sector
  sector = tick.info['sector']
  wks.update_cell(index + 2, 14, sector)
  
  #Industry
  industry = tick.info['industry']
  wks.update_cell(index + 2, 15, industry)

    2022-08-17                        Dividendo  BRIT AMER TOBACC RG   76  \
2   2022-08-17                            Venta  ADVANCED EMISSIO RG  240   
3   2022-08-16                        Dividendo      SPDR GL DIV USD   65   
4   2022-08-15                           Compra           NAGARRO RG   16   
5   2022-08-15                            Venta              INDITEX   97   
6   2022-07-27   Traspaso de efectivo de salida                         0   
7   2022-07-22                        Dividendo      HOLLAND COLOURS   15   
8   2022-07-15                        Dividendo  THERMO FISHER SC RG    4   
9   2022-07-15                        Dividendo        ISHSEU600UTIL   55   
10  2022-07-14  Traspaso de efectivo de entrada                         0   
11  2022-07-14                           Compra   ISHARES ASIA PACIF  100   
12  2022-07-14                           Compra            AHOLD DEL  115   
13  2022-06-29                        Dividendo      ISHS EM DIV USD  131   

KeyError: ignored